In [14]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [18]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-23 06:56:30.577499


In [19]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [20]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [21]:
current_date.to_datetime64()

numpy.datetime64('2025-03-23T06:00:00.000000000')

In [23]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [24]:
fetch_data_to

Timestamp('2025-03-23 06:00:00+0000', tz='UTC')

In [26]:
fetch_data_from

Timestamp('2025-02-22 06:00:00+0000', tz='UTC')

In [46]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks and remove timezone info
    historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
    historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from['pickup_datetime'] = rides_from['pickup_datetime'].dt.tz_localize(None)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = rides_to['pickup_datetime'].dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    rides['pickup_datetime'] += timedelta(weeks=52)
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [47]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [48]:
rides

,pickup_datetime,pickup_location_id
1523756,2025-03-08 18:11:37,2
1556605,2025-03-08 23:16:48,2
2343561,2025-03-16 17:46:48,2
2557550,2025-03-19 00:27:26,2
2599822,2025-03-19 14:43:26,2
...,...,...
2987787,2025-03-23 05:39:38,263
2987763,2025-03-23 05:49:20,263
2987769,2025-03-23 05:50:04,263
3295194,2025-03-23 05:52:41,263


In [49]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [50]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-22 06:00:00,2,0
1,2025-02-22 07:00:00,2,0
2,2025-02-22 08:00:00,2,0
3,2025-02-22 09:00:00,2,0
4,2025-02-22 10:00:00,2,0
...,...,...,...
176083,2025-03-23 01:00:00,263,66
176084,2025-03-23 02:00:00,263,38
176085,2025-03-23 03:00:00,263,23
176086,2025-03-23 04:00:00,263,18


In [51]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176088 entries, 0 to 176087
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176088 non-null  datetime64[ns]
 1   pickup_location_id  176088 non-null  int16         
 2   rides               176088 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [52]:
# import hopsworks

# # connect to the project
# project = hopsworks.login(
#     project=config.HOPSWORKS_PROJECT_NAME,
#     api_key_value=config.HOPSWORKS_API_KEY
# )

# # connect to the feature store
# feature_store = project.get_feature_store()

# # connect to the feature group
# feature_group = feature_store.get_feature_group(
#     name=config.FEATURE_GROUP_NAME,
#     version=config.FEATURE_GROUP_VERSION,
# )

import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-03-23 03:28:41,778 INFO: Initializing external client
2025-03-23 03:28:41,779 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 03:28:42,789 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220760


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1220760/featurestores/1208392/featuregroups/time_series_hourly_feature_group). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: time_series_hourly_feature_group feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: time_series_hourly_feature_group feature group version: 1

In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})